<a href="https://colab.research.google.com/github/barcode0725/tugassistemrekomendasi/blob/main/SistemRekomendasi0018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Content Based Filtering : Rekomendasi Modul Pembelajaran Dari Program Magang Dan Studi Independen Kampus Merdeka

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import random

df = pd.read_csv("https://raw.githubusercontent.com/barcode0725/tugassistemrekomendasi/main/a.csv")
df.head()

,Mitra,Alamat,Modul
0,Dwi Rahma Firmansyah,Jln Rorojonggrang Timur VIII RT9 RW9,Implementasi Basis Data Web Service Sistem Bas...
1,PT. IlmuKomputerCom Braindevs Sistema,"Menara Bidakara 1 Lantai 2, Jl. Gatot Subroto ...",Critical Thinking Team Work Communication Skil...
2,PT Sembilan Matahari,"Jl. Muararajeun No. 26 RT 005 RW 011, Cihaurge...",Kemampuan Management Data Asset Pencarian refe...
3,PT Telkom Indonesia (Persero) Tbk,"Jl Japati 1, Bandung",Content Review Job Performance Strategic Commu...
4,PT Abbauf Mulia Konsultan Teknologi,"APL Tower Central Park Lantai 19 Unit T7, Jl. ...",Memonitoring Kerjasama Kerjasama Tim Secara Di...


## 1. Ikhtisar

In [ ]:
df.describe()

,Mitra,Alamat,Modul
count,13,13,13
unique,13,13,13
top,Dwi Rahma Firmansyah,Jln Rorojonggrang Timur VIII RT9 RW9,Implementasi Basis Data Web Service Sistem Bas...
freq,1,1,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Mitra   13 non-null     object
 1   Alamat  13 non-null     object
 2   Modul   13 non-null     object
dtypes: object(3)
memory usage: 440.0+ bytes


## 2. Tampilan Modul Pembelajaran (Sebelum Preprocessing)

In [ ]:
def print_description(index):
    example = df[df.index == index][['Modul', 'Mitra','Alamat']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Mitra:', example[1])
        print('Alamat:', example[2])

In [ ]:
print_description(1)

Critical Thinking Team Work Communication Skills Time Management Requirement Analysis
Mitra: PT. IlmuKomputerCom Braindevs Sistema
Alamat: Menara Bidakara 1 Lantai 2, Jl. Gatot Subroto Kav. 71-73 Pancoran Jakarta Selatan 12870


In [ ]:
print_description(11)

Basic Frontend Web Development With HTML & CSS: Memahami penulisan HTML dan penggunaan CSS dalam Web Development dasar
Mitra: PT Ruang Raya Indonesia
Alamat: l. Dr. Saharjo No.161, Manggarai Selatan, Tebet, Kota Jakarta Selatan, Daerah Khusus Ibukota Jakarta 12860


In [ ]:
print_description(0)

Implementasi Basis Data Web Service Sistem Basis Data Pemrograman Web Jaringan Komputer Sistem Operasi Pemrograman Aplikasi Mobile Text Mining
Mitra: Dwi Rahma Firmansyah
Alamat: Jln Rorojonggrang Timur VIII RT9 RW9


## 3. Text Preprocessing

In [ ]:
import nltk
nltk.download('stopwords')
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
#stopworda = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    #text = ' '.join(word for word in text.split() if word not in stopworda) # hapus stopword dari kolom deskripsi
    return text
    
df['modul_pembelajaran_clean'] = df['Modul'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df.head()

,Mitra,Alamat,Modul,modul_pembelajaran_clean
0,Dwi Rahma Firmansyah,Jln Rorojonggrang Timur VIII RT9 RW9,Implementasi Basis Data Web Service Sistem Bas...,implementasi basis data web service sistem bas...
1,PT. IlmuKomputerCom Braindevs Sistema,"Menara Bidakara 1 Lantai 2, Jl. Gatot Subroto ...",Critical Thinking Team Work Communication Skil...,critical thinking team work communication skil...
2,PT Sembilan Matahari,"Jl. Muararajeun No. 26 RT 005 RW 011, Cihaurge...",Kemampuan Management Data Asset Pencarian refe...,kemampuan management data asset pencarian refe...
3,PT Telkom Indonesia (Persero) Tbk,"Jl Japati 1, Bandung",Content Review Job Performance Strategic Commu...,content review job performance strategic commu...
4,PT Abbauf Mulia Konsultan Teknologi,"APL Tower Central Park Lantai 19 Unit T7, Jl. ...",Memonitoring Kerjasama Kerjasama Tim Secara Di...,memonitoring kerjasama kerjasama tim secara di...


## 4. Tampilan Modul Pembelajaran (Setelah Preprocessing)

In [ ]:
# Deskripsi kedua (Setelah preprocessing)
def print_description_clean(index):
    example = df[df.index == index][['modul_pembelajaran_clean', 'Mitra','Alamat']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Mitra:', example[1])
        print('Alamat:', example[2])

In [ ]:
print_description_clean(1)

critical thinking team work communication skills time management requirement analysis
Mitra: PT. IlmuKomputerCom Braindevs Sistema
Alamat: Menara Bidakara 1 Lantai 2, Jl. Gatot Subroto Kav. 71-73 Pancoran Jakarta Selatan 12870


In [ ]:
print_description_clean(11)

basic frontend web development with html  css memahami penulisan html dan penggunaan css dalam web development dasar
Mitra: PT Ruang Raya Indonesia
Alamat: l. Dr. Saharjo No.161, Manggarai Selatan, Tebet, Kota Jakarta Selatan, Daerah Khusus Ibukota Jakarta 12860


In [ ]:
print_description(0)

Implementasi Basis Data Web Service Sistem Basis Data Pemrograman Web Jaringan Komputer Sistem Operasi Pemrograman Aplikasi Mobile Text Mining
Mitra: Dwi Rahma Firmansyah
Alamat: Jln Rorojonggrang Timur VIII RT9 RW9


## 5. TF-IDF & Cosine Similarity

In [ ]:
df.set_index('Mitra', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['modul_pembelajaran_clean'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.        , 0.02171863, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.02947463, 0.0410139 , 0.0224564 ],
       [0.        , 1.        , 0.01703802, 0.02251484, 0.        ,
        0.        , 0.10007079, 0.        , 0.        , 0.02094631,
        0.        , 0.        , 0.        ],
       [0.02171863, 0.01703802, 1.        , 0.05220856, 0.13439448,
        0.01676363, 0.        , 0.00451357, 0.05657502, 0.        ,
        0.00708765, 0.0062711 , 0.        ],
       [0.        , 0.02251484, 0.05220856, 1.        , 0.        ,
        0.        , 0.01292289, 0.        , 0.        , 0.01302472,
        0.        , 0.        , 0.01095437],
       [0.        , 0.        , 0.13439448, 0.        , 1.        ,
        0.01437944, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.01676363, 0.        , 0.01437944,
        1.        , 0.     

In [ ]:
# Set index utama di kolom 'name'
indices = pd.Series(df.index)
indices[:50]

0                      Dwi Rahma Firmansyah
1     PT. IlmuKomputerCom Braindevs Sistema
2                      PT Sembilan Matahari
3         PT Telkom Indonesia (Persero) Tbk
4       PT Abbauf Mulia Konsultan Teknologi
5              PT Baracipta Esa Engineering
6                   PT Global Tiket Network
7              PT Semesta Integrasi Digital
8             PT STECHOQ ROBOTIKA INDONESIA
9                    Terra Weather Pte. Ltd
10               PT Orbit Ventura Indonesia
11                  PT Ruang Raya Indonesia
12             PT Nurul Fikri Cipta Inovasi
Name: Mitra, dtype: object

## 6. Modelling

In [ ]:
def recommendations(name, cos_sim = cos_sim):
    
    recommended_modul = []
    
    # Mengambil nama modul berdasarkan variabel indicies
    idx = indices[indices == name].index[0]

    # Membuat series berdasarkan skor kesamaan
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # mengambil index dan dibuat 3 baris rekomendasi terbaik
    top_10_indexes = list(score_series.iloc[1:4].index)
    
    for i in top_10_indexes:
        recommended_modul.append(list(df.index)[i])
        
    return recommended_modul

## 7. Prediksi

In [ ]:
recommendations('Dwi Rahma Firmansyah')

['PT Ruang Raya Indonesia',
 'PT Orbit Ventura Indonesia',
 'PT Nurul Fikri Cipta Inovasi']

In [ ]:
recommendations("PT Ruang Raya Indonesia")

['Dwi Rahma Firmansyah',
 'PT Semesta Integrasi Digital',
 'PT Nurul Fikri Cipta Inovasi']

In [ ]:
recommendations("PT Abbauf Mulia Konsultan Teknologi")

['PT Sembilan Matahari',
 'PT Baracipta Esa Engineering',
 'Dwi Rahma Firmansyah']